In [7]:
import cv2
import os
import numpy as np
import gc
from mtcnn import MTCNN
from tqdm import tqdm

In [6]:
!pip install pyOpenSSL==23.2.0

  Attempting uninstall: pyOpenSSL
    Found existing installation: pyOpenSSL 21.0.0
    Uninstalling pyOpenSSL-21.0.0:
      Successfully uninstalled pyOpenSSL-21.0.0


In [4]:
!pip install setuptools==65.5.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
google-api-core 1.25.1 requires google-auth<2.0dev,>=1.21.1, but you have google-auth 2.38.0 which is incompatible.
google-cloud-core 1.7.1 requires google-auth<2.0dev,>=1.24.0, but you have google-auth 2.38.0 which is incompatible.
google-cloud-storage 1.31.0 requires google-auth<2.0dev,>=1.11.0, but you have google-auth 2.38.0 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.3 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.



   ---------------------------------------- 1.2/1.2 MB 175.0 kB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 76.1.0
    Uninstalling setuptools-76.1.0:
      Successfully uninstalled setuptools-76.1.0


In [2]:
!pip install mtcnn

   ---------------------------------------- 2.3/2.3 MB 114.3 kB/s eta 0:00:00


In [2]:
# Initialize the MTCNN face detector
detector = MTCNN()

In [3]:
def crop_faces_mtcnn_hair_fix(input_folder, output_folder, scale_w=1.5, scale_h=1.45, hair_extension_factor=0.15, max_size=800):
    """
    Detects faces using MTCNN and expands the bounding box to ensure both ears and full hair are visible,
    while keeping the neck cropped perfectly (no clothing).

    :param input_folder: Path to dataset folder containing subfolders with images.
    :param output_folder: Path to save cropped face images, maintaining subfolder structure.
    :param scale_w: Factor to expand bounding box width (default = 1.5).
    :param scale_h: Factor to expand bounding box height (default = 1.45).
    :param hair_extension_factor: Factor to extend the upper part for hair visibility (default = 0.15).
    :param max_size: Maximum image width/height to prevent memory overload (default = 800 pixels).
    """
    for subfolder in os.listdir(input_folder):
        subfolder_path = os.path.join(input_folder, subfolder)
        if not os.path.isdir(subfolder_path):
            continue  # Skip if not a directory

        output_subfolder = os.path.join(output_folder, subfolder)
        os.makedirs(output_subfolder, exist_ok=True)

        image_count = 1  # Counter for naming images

        for filename in tqdm(os.listdir(subfolder_path), desc=f"Processing {subfolder}"):
            if not filename.endswith(('.jpg', '.jpeg', '.png')):
                continue

            image_path = os.path.join(subfolder_path, filename)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Error loading {filename}, skipping...")
                continue

            # Resize large images to prevent memory errors
            h, w, _ = image.shape
            if max(h, w) > max_size:
                scale_factor = max_size / max(h, w)
                image = cv2.resize(image, (int(w * scale_factor), int(h * scale_factor)))

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Detect faces (setting `min_face_size` to 40 to filter out tiny faces)
            faces = detector.detect_faces(image_rgb)

            if faces:
                for face in faces:
                    x, y, w_bbox, h_bbox = face['box']

                    # Expand width equally on both sides for ears
                    new_w = int(w_bbox * scale_w)
                    x = x - int((new_w - w_bbox) // 2)

                    # Expand height but only add to the top for hair
                    new_h = int(h_bbox * scale_h)
                    y = y - int(h_bbox * hair_extension_factor)

                    # Ensure the box stays within the image boundaries
                    x = max(0, x)
                    y = max(0, y)
                    new_w = min(new_w, image.shape[1] - x)
                    new_h = min(new_h, image.shape[0] - y)

                    # Crop the expanded face region
                    face_crop = image[y:y + new_h, x:x + new_w]

                    # Save cropped image
                    new_filename = f"{subfolder}_{image_count:02d}.jpg"
                    output_path = os.path.join(output_subfolder, new_filename)
                    cv2.imwrite(output_path, face_crop)
                    image_count += 1  # Increment counter
                    break  # Only save the first detected face

            else:
                print(f"No face detected in {filename}")

            # Clear memory after processing each image
            del image, image_rgb, faces
            gc.collect()

In [4]:

# Example Usage
input_folder = r"sanjana"
output_folder = r"cv"

crop_faces_mtcnn_hair_fix(input_folder, output_folder, scale_w=1.5, scale_h=1.45, hair_extension_factor=0.15)


Processing Amit Shah:  11%|██████▋                                                    | 43/380 [00:14<01:46,  3.18it/s]

No face detected in Amit Shah_141.jpg


Processing Amit Shah:  19%|███████████▎                                               | 73/380 [00:24<01:45,  2.91it/s]

Error loading Amit Shah_173.jpg, skipping...


Processing Amit Shah:  34%|███████████████████▋                                      | 129/380 [00:40<01:12,  3.49it/s]

...

Processing Yogi Adityanath:  86%|████████████████████████████████████████████▍       | 320/374 [01:42<00:15,  3.42it/s]

Error loading Yogi Adityanath_42.jpg, skipping...


Processing Yogi Adityanath:  87%|█████████████████████████████████████████████▎      | 326/374 [01:44<00:16,  3.00it/s]

Error loading Yogi Adityanath_5.jpg, skipping...


Processing Yogi Adityanath: 100%|████████████████████████████████████████████████████| 374/374 [02:00<00:00,  3.10it/s]
